In [0]:
print("Healthy")

In [0]:

from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector


pd.set_option('display.max_columns', None)


import geopy
from geopy.geocoders import ArcGIS, Nominatim

import warnings
warnings.filterwarnings('ignore')

import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)

host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")

pd.set_option('display.float_format', lambda x: '%.2f' % x)

from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
print(today)
print(yesterday)



In [0]:

last_2_wks = datetime.today() - timedelta(days = 14)
last_2_wks = last_2_wks.strftime('%Y-%m-%d')
print('------------------------------------')
print(last_2_wks)

In [0]:
lac = pd.read_sql_query(f'''
            select
            max(account_id) as max_id,
            max(activation_date) as max_date
            from dwh_all_accounts
            ''', conn)
last_acct = lac['max_id'][0], '-', lac['max_date'][0]
last_acct[0], last_acct[2]

In [0]:
%%time
all_accts = pd.read_sql_query(f'''
                select
                ifnull(mc.id, mg.id) client_id,
                -- ifnull(mc.display_name, mg.display_name) as client_name,
                sa.id account_id,
                sa.account_no,
                sp.name as product_name,
                sp.id as product_id,
                case when isnull(mc.id) then 'joint account' else 'standard account' end as 'account_category',
                case when isnull(sa.closedon_date) then 'active' else 'closed' end as 'account_status',
                sa.approvedon_date as activation_date,
                sa.closedon_date
                from m_savings_account sa
                left join m_client mc on mc.id = sa.client_id
                left join m_group mg on mg.id = sa.group_id
                join m_savings_product sp on sp.id = sa.product_id
                where sa.id > {last_acct[0]}
                ''', Mifosdb)

all_accts

In [0]:
all_accts.shape

In [0]:
%%time
all_accts.to_sql('dwh_all_accounts', conn, index=False, if_exists='append', chunksize= 10000, method = 'multi')

## Product types ETL

In [0]:
product = pd.read_sql_query(f'''
        select
        id product_id,
        name as product_name,
        short_name,
        description
        from m_savings_product 
        ''', Mifosdb)

product

In [0]:
%%time
product.to_sql('dwh_product_types', conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')

In [0]:
import datetime

last_run_date = datetime.datetime.now()
print("Last run date:", last_run_date)
